In [2]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
from segmentation import segment_table_markers_hsv
from utils import show, select_roi, get_color_range, get_image_from_camera
import pickle
from kas_camera_utils import stream
from params import top_camera_id, front_camera_id

In [ ]:
# calibrate segmentation roi

read_images_from_files = True
if read_images_from_files:
    top_image_file = "images/segment_scene/top.jpg"
    front_image_file = "images/segment_scene/front.jpg"
    top_image = cv2.imread(top_image_file)
    front_image = cv2.imread(front_image_file)
else:
    top_image = get_image_from_camera(top_camera_id)
    front_image = get_image_from_camera(front_camera_id)

top_working_area_roi = select_roi(top_image, full_by_default=True, window_name="working area roi")
top_goal_and_stop_line_roi = select_roi(top_image, full_by_default=True, window_name="goal and stop line roi")

front_working_area_roi = select_roi(front_image, full_by_default=True, window_name="working area roi")
front_goal_and_stop_line_roi = select_roi(front_image, full_by_default=True, window_name="goal and stop line roi")

top_roi = {"working_area": top_working_area_roi, "goal_and_stop_line": top_goal_and_stop_line_roi}
front_roi = {"working_area": front_working_area_roi, "goal_and_stop_line": front_goal_and_stop_line_roi}
roi = {"top": top_roi, "front": front_roi}

print(f"Working area roi:")
print(f"    top: x_range, y_range = {top_working_area_roi}")
print(f"    front: x_range, y_range = {front_working_area_roi}")
print(f"Goal and stop line roi:")
print(f"    top: x_range, y_range = {top_goal_and_stop_line_roi}")
print(f"    front: x_range, y_range = {front_goal_and_stop_line_roi}")

In [ ]:
output_file = "data/segmentation_roi.pickle"
with open(output_file, 'wb') as f:
    pickle.dump(roi, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# segment table markers

read_image_from_file = True
if read_image_from_file:
    view = "top"
    image_file = "images/segmentation/0000.jpg"
    image = cv2.imread(image_file)
else:
    view = "top"
    image = get_image_from_camera(top_camera_id)

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
(refined_mask, orig_mask), polygons = segment_table_markers_hsv(hsv, view=view)
show(refined_mask)